### This notebook is intended to predict sql injection attacks
    Total data  : 5416
    Benign data : 3072 + 476 = 3548
    SQLI Data   : 1128 + 740 = 1868
    
#### At the end of notebook are results of testing

In [1]:

# import required packages

import glob
import time
import pandas as pd
# from xml.dom import minidom
from nltk import ngrams
from nltk.tokenize import sent_tokenize
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem import PorterStemmer
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /home/ebryx/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/ebryx/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/ebryx/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
import keras

Using TensorFlow backend.


In [3]:
import nltk
import pandas as pd
import os

### File_name: git_foospidy_payloads_fuzzing_code_db_sqli.txt
##### Comments :  
1. removed \n at the end of each line
2.  replaced %20 by space
#### Source: https://github.com/foospidy/payloads/tree/master/other/sqli

In [4]:
# preprocess sql data to have same format for all files

def clean_sqli_data(data):
    
    for i in range(len(data)):
        
        data[i]=data[i].replace('\n', '')
        data[i]=data[i].replace('%20', ' ')
        data[i]=data[i].replace('=', ' = ')
        data[i]=data[i].replace('((', ' (( ')
        data[i]=data[i].replace('))', ' )) ')
        data[i]=data[i].replace('(', ' ( ')
        data[i]=data[i].replace(')', ' ) ')
    
    return data

In [5]:
path='./data/collection/git_foospidy_payloads_fuzzing_code_db_sqli.txt'

In [6]:
# read data from file

sql_lines_fuzzing=[]
f = open(path, "r")
for x in f:
    sql_lines_fuzzing.append(x)

In [7]:
sql_lines_fuzzing=clean_sqli_data(sql_lines_fuzzing) 

In [8]:
sql_lines_fuzzing[:15]

["'sqlvuln",
 "'+sqlvuln",
 'sqlvuln;',
 ' ( sqlvuln ) ',
 "a' or 1 = 1--",
 '"a"" or 1 = 1--"',
 ' or a  =  a',
 "a' or 'a'  =  'a",
 '1 or 1 = 1',
 "a' waitfor delay '0:0:10'--",
 "1 waitfor delay '0:0:10'--",
 'declare @q nvarchar  ( 4000 )  select @q  = ',
 '0x770061006900740066006F0072002000640065006C00610079002000270030003A0030003A',
 '0',
 '031003000270000']

### File_Name: git_foospidy_payloads_other_sqli_camoufl4g3.txt
##### Comments : ok
#### Source: https://github.com/foospidy/payloads/tree/master/other/sqli

In [9]:
path='./data/collection/git_foospidy_payloads_other_sqli_camoufl4g3.txt'

In [10]:
# read data from file

sql_lines_camoufl4g3=[]
f = open(path, "r")
for x in f:
    sql_lines_camoufl4g3.append(x)

In [11]:
sql_lines_camoufl4g3[:15] # data before cleaning

["'-'\n",
 "' '\n",
 "'&'\n",
 "'^'\n",
 "'*'\n",
 "' or ''-'\n",
 "' or '' '\n",
 "' or ''&'\n",
 "' or ''^'\n",
 "' or ''*'\n",
 '"-"\n',
 '" "\n',
 '"&"\n',
 '"^"\n',
 '"*"\n']

In [12]:
sql_lines_camoufl4g3=clean_sqli_data(sql_lines_camoufl4g3)

In [13]:
sql_lines_camoufl4g3[:15]  # data after cleaning

["'-'",
 "' '",
 "'&'",
 "'^'",
 "'*'",
 "' or ''-'",
 "' or '' '",
 "' or ''&'",
 "' or ''^'",
 "' or ''*'",
 '"-"',
 '" "',
 '"&"',
 '"^"',
 '"*"']

### File_Name: git_foospidy_payloads_other_sqli_libinjection-bypasses.txt
##### Comments: Should Remove &()o1: from start of each sentence
#### Source: https://github.com/foospidy/payloads/tree/master/other/sqli

In [14]:
path='./data/collection/git_foospidy_payloads_other_sqli_libinjection-bypasses.txt'

In [15]:
# read data from file

sql_lines_bypasses=[]
f = open(path, "r")
for x in f:
    sql_lines_bypasses.append(x)

In [16]:
sql_lines_bypasses[:5] # before cleaning

['&()o1: select * from users where id=1 or (\\)=1 union select 1,@@VERSION -- 1\n',
 '&(.)o: select * from users where id=1 or (\\.)=1 union select 1,@@VERSION -- 1\n',
 '&(1&1: select * from users where id=1 or (\\+)=1 or 1=1 -- 1\n',
 '&(1)o: select * from users where id=1 or (1)=1 union select 1,banner from v$version where rownum=1 -- 1\n',
 '&(1UE: select * from users where id=1 or (\\+)=1 union select 1,@@VERSION -- 1\n']

In [17]:
sql_lines_bypasses=clean_sqli_data(sql_lines_bypasses)

In [18]:
sql_lines_bypasses[:5]  # data after cleaning

['& (  ) o1: select * from users where id = 1 or  ( \\ )  = 1 union select 1,@@VERSION -- 1',
 '& ( . ) o: select * from users where id = 1 or  ( \\. )  = 1 union select 1,@@VERSION -- 1',
 '& ( 1&1: select * from users where id = 1 or  ( \\+ )  = 1 or 1 = 1 -- 1',
 '& ( 1 ) o: select * from users where id = 1 or  ( 1 )  = 1 union select 1,banner from v$version where rownum = 1 -- 1',
 '& ( 1UE: select * from users where id = 1 or  ( \\+ )  = 1 union select 1,@@VERSION -- 1']

In [19]:
# if don't want &(*)* sign in beginning of each sentence then run next code else don't

for i in range(len(sql_lines_bypasses)):
    sentence=sql_lines_bypasses[i]
    sql_lines_bypasses[i]=sentence.split(':')[1]
    

In [20]:
sql_lines_bypasses[:5]

[' select * from users where id = 1 or  ( \\ )  = 1 union select 1,@@VERSION -- 1',
 ' select * from users where id = 1 or  ( \\. )  = 1 union select 1,@@VERSION -- 1',
 ' select * from users where id = 1 or  ( \\+ )  = 1 or 1 = 1 -- 1',
 ' select * from users where id = 1 or  ( 1 )  = 1 union select 1,banner from v$version where rownum = 1 -- 1',
 ' select * from users where id = 1 or  ( \\+ )  = 1 union select 1,@@VERSION -- 1']

### File_Name: git_foospidy_payloads_owasp_sqli.txt
##### Comments: ok
#### Source: https://github.com/foospidy/payloads/tree/master/other/sqli

In [21]:
path='./data/collection/git_foospidy_payloads_owasp_sqli.txt'

In [22]:
# read data from file

sql_lines_owasp=[]
f = open(path, "r")
for x in f:
    sql_lines_owasp.append(x)

In [23]:
sql_lines_owasp[0:15] # before cleaning

['a\n',
 "a' \n",
 "a' --\n",
 "a' or 1=1; --\n",
 '@\n',
 '?\n',
 "' and 1=0) union all\n",
 '? or 1=1 --\n',
 "x' and userid is NULL; --\n",
 "x' and email is NULL; --\n",
 "anything' or 'x'='x\n",
 "x' and 1=(select count(*) from tabname); --\n",
 "x' and members.email is NULL; --\n",
 "x' or full_name like '%bob%\n",
 '23 or 1=1; --\n']

In [24]:
sql_lines_owasp=clean_sqli_data(sql_lines_owasp)

In [25]:
sql_lines_owasp[:15] # after cleaning

['a',
 "a' ",
 "a' --",
 "a' or 1 = 1; --",
 '@',
 '?',
 "' and 1 = 0 )  union all",
 '? or 1 = 1 --',
 "x' and userid is NULL; --",
 "x' and email is NULL; --",
 "anything' or 'x' = 'x",
 "x' and 1 =  ( select count ( * )  from tabname ) ; --",
 "x' and members.email is NULL; --",
 "x' or full_name like '%bob%",
 '23 or 1 = 1; --']

### File_Name: git_seclist_fuzzing_sqli_Generic-SQLi.txt
##### comments: should we remove %20 by spaces because %20 equals to space
#### Source: https://github.com/danielmiessler/SecLists/tree/master/Fuzzing/SQLi

In [26]:
path='./data/collection/git_seclist_fuzzing_sqli_Generic-SQLi.txt'

In [27]:
# read data from file

sql_lines_Generic=[]
f = open(path, "r")
for x in f:
    sql_lines_Generic.append(x)

In [28]:
sql_lines_Generic[:15] # before cleaning

[")%20or%20('x'='x\n",
 '%20or%201=1\n',
 "; execute immediate 'sel' || 'ect us' || 'er'\n",
 'benchmark(10000000,MD5(1))#\n',
 'update\n',
 '";waitfor delay \'0:0:__TIME__\'--\n',
 '1) or pg_sleep(__TIME__)--\n',
 '||(elt(-3+5,bin(15),ord(10),hex(char(45))))\n',
 '"hi"") or (""a""=""a"\n',
 'delete\n',
 'like\n',
 '" or sleep(__TIME__)#\n',
 'pg_sleep(__TIME__)--\n',
 '*(|(objectclass=*))\n',
 'declare @q nvarchar (200) 0x730065006c00650063 ...\n']

In [29]:
sql_lines_Generic=clean_sqli_data(sql_lines_Generic)

In [30]:
sql_lines_Generic[:15] # after cleaning

[" )  or  ( 'x' = 'x",
 ' or 1 = 1',
 "; execute immediate 'sel' || 'ect us' || 'er'",
 'benchmark ( 10000000,MD5 ( 1  )  )  #',
 'update',
 '";waitfor delay \'0:0:__TIME__\'--',
 '1 )  or pg_sleep ( __TIME__ ) --',
 '|| ( elt ( -3+5,bin ( 15 ) ,ord ( 10 ) ,hex ( char ( 45  )  )    )  )  ',
 '"hi"" )  or  ( ""a"" = ""a"',
 'delete',
 'like',
 '" or sleep ( __TIME__ ) #',
 'pg_sleep ( __TIME__ ) --',
 '* ( | ( objectclass = *  )  )  ',
 'declare @q nvarchar  ( 200 )  0x730065006c00650063 ...']

### File_name: scottparker_ml_sqli_src__trainingdata_plain.txt
##### Comments: 
removed last 22 records containing URLs
#### Source: https://github.com/tungpv98/Detect-Sql-Injection-by-Machine-Learning/tree/ebeb3287931677a3a42f11a7a08dbc13e374ff05/Sql-Injection/source/trainingdata

In [31]:

# function to remove stopwords

stop_words = set(stopwords.words('english')) 

def fun_remove_stop_words(posts):

    filtered=''
    
    for x in posts.split(' '):
        if x not in stop_words:
            filtered+=' '+x
    
    return filtered

In [32]:
path='./data/collection/'
file="scottparker_ml_sqli_src__trainingdata_plain.txt"

In [33]:
#read benign data

df = pd.read_csv(os.path.join(path,file), sep='Aw3s0meSc0t7', names=['benign'], header=None, engine='python')

In [34]:
df.head()

,benign
0,Add plain text here
1,Ne te quaesiveris extra.
2,Man is his own star; and the soul that can
3,"Render an honest and a perfect man,"
4,"Commands all light, all influence, all fate;"


In [35]:
plain_text=df['benign'].values  # get sentences

In [36]:
plain_text[:5]

array(['Add plain text here', 'Ne te quaesiveris extra.',
       'Man is his own star; and the soul that can',
       'Render an honest and a perfect man,',
       'Commands all light, all influence, all fate;'], dtype=object)

In [37]:
plain_text=plain_text[:-22] # removed last 22 records that were urls

In [38]:
len(plain_text)

3672

In [39]:
# convert from list to string

data=''
for x in plain_text:
    data+=" " + x

In [40]:
type(data)

str

In [41]:
data=fun_remove_stop_words(data)  # remove stop words
data=data.split('.')              # split sentences

In [42]:
# seperate words inside tags

for i in range(len(data)):
    data[i]=data[i].replace('<', ' <')
    data[i]=data[i].replace('>', '> ')
    data[i]=data[i].replace('=', ' = ')

In [43]:
data[:5]

['  Add plain text Ne te quaesiveris extra',
 ' Man star; soul Render honest perfect man, Commands light, influence, fate; Nothing falls early late',
 ' Our acts angels are, good ill, Our fatal shadows walk us still',
 ' Epilogue Beaumont Fletcher’s Honest Man’s Fortune Cast bantling rocks, Suckle she-wolf’s teat; Wintered hawk fox, Power speed hands feet',
 ' I read day verses written eminent painter original conventional']

Statistics

In [44]:
print("Benign records: %2i" %len(data))

Benign records: 3072


In [45]:
# read self created benign data

path='./data/collection/benign_for_training.txt'
benign_data=[]
f = open(path, "r")
for x in f:
    benign_data.append(x)


In [49]:
# read self created sqli data

path='./data/collection/sqli_for_training.txt'
sqli_data=[]
f = open(path, "r")
for x in f:
    sqli_data.append(x)


In [50]:
len(benign_data)

440

In [51]:
len(sqli_data)

290

In [52]:
print(f"SQL fuzzing : {len(sql_lines_fuzzing)}  camoufl4gs : {len(sql_lines_camoufl4g3)} parsed : {len(sql_lines_bypasses)} owasp : {len(sql_lines_owasp)} generic : {len(sql_lines_Generic)} \n total sql injection data : {len(sql_lines_Generic)+len(sql_lines_owasp)+len(sql_lines_bypasses)+len(sql_lines_camoufl4g3)+len(sql_lines_fuzzing)} ")

SQL fuzzing : 142  camoufl4gs : 77 parsed : 474 owasp : 167 generic : 268 
 total sql injection data : 1128 


#### combination of all sqli attacks

In [53]:
all_sqli_sentence=sql_lines_owasp+sql_lines_bypasses+sql_lines_camoufl4g3+sql_lines_fuzzing+sql_lines_Generic

In [54]:
len(all_sqli_sentence)

1128

In [55]:
# replace numeric values by a keyword 'numeric'
def optional_numeric_to_numeric(all_sqli_sentence):
    
    for i in range(len(all_sqli_sentence)):
        
        all_sqli_sentence[i]=all_sqli_sentence[i].replace('1 ', 'numeric')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace(' 1', 'numeric')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace("'1 ", "'numeric ")
        all_sqli_sentence[i]=all_sqli_sentence[i].replace(" 1'", " numeric'")
        all_sqli_sentence[i]=all_sqli_sentence[i].replace('1,', 'numeric,')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace("1\ ", 'numeric')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace("‘1", '‘numeric')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace(" 2 ", " numeric ")
        all_sqli_sentence[i]=all_sqli_sentence[i].replace(' 3 ', ' numeric ')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace(' 3--', ' numeric--')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace(" 4 ", ' numeric ')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace(" 5 ", ' numeric ')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace(' 6 ', ' numeric ')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace(" 7 ", ' numeric ')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace(" 8 ", ' numeric ')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace('1234', ' numeric ')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace("22", ' numeric ')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace(" 8 ", ' numeric ')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace(" 200 ", ' numeric ')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace("23 ", ' numeric ')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace('"1', '"numeric')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace('1"', '"numeric')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace("7659", 'numeric')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace(" 37 ", ' numeric ')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace(" 45 ", ' numeric ')
    
    return all_sqli_sentence


In [56]:
all_sqli_sentence

['a',
 "a' ",
 "a' --",
 "a' or 1 = 1; --",
 '@',
 '?',
 "' and 1 = 0 )  union all",
 '? or 1 = 1 --',
 "x' and userid is NULL; --",
 "x' and email is NULL; --",
 "anything' or 'x' = 'x",
 "x' and 1 =  ( select count ( * )  from tabname ) ; --",
 "x' and members.email is NULL; --",
 "x' or full_name like '%bob%",
 '23 or 1 = 1; --',
 "'; exec master..xp_cmdshell 'ping 172.10.1.255'--",
 'a',
 '1 or 1 = 1',
 "1' or '1' = '1",
 "1 and user_name (  )   =  'dbo'",
 '1',
 "1'1",
 '1 exec sp_  ( or exec xp_ ) ',
 '1 and 1 = 1',
 "1' and 1 =  ( select count ( * )  from tablenames ) ; --",
 '1',
 "1 and user_name (  )   =  'dbo'",
 "\\'; desc users; --",
 "1\\'1",
 "1' and non_existant_table  =  '1",
 "' or username is not NULL or username  =  '",
 "1 and ascii ( lower ( substring  (  (  select top 1 name from sysobjects where xtype = 'u' ) , 1, 1  )  )   )  > 116",
 "1 union all select 1,2,3,4,5,6,name from sysobjects where xtype  =  'u' --",
 '1 uni/**/on select all from where',
 '’ or ‘1’ =

#### Combination of benign an attack data

In [57]:
import pandas as pd

In [58]:
# give labels to sql data

values=[]
for i in all_sqli_sentence:
    values.append((i,1))

In [59]:
# give labels to benign data

for i in data:
    values.append((i,0))

In [60]:
len(all_sqli_sentence)+len(data)

4200

In [61]:
len(values)

4200

In [62]:
for i in benign_data:
    values.append((i,0))

In [63]:
len(values)

4640

In [64]:
for i in sqli_data:
    values.append((i,1))

In [65]:
len(values)

4930

In [66]:
values[1]

("a' ", 1)

In [67]:
# convert to dataframe

df=pd.DataFrame(values,columns=['Sentence','Label'])



In [68]:
df.head()

,Sentence,Label
0,a,1
1,a',1
2,a' --,1
3,a' or 1 = 1; --,1
4,@,1


### Save data as csv

In [69]:
df.to_csv('sqli.csv', index=False, encoding='utf-16')

In [70]:
df=pd.read_csv('sqli.csv',encoding='utf-16')

In [71]:
# vectorization of data

from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer( min_df=2, max_df=0.7, stop_words=stopwords.words('english'))
posts = vectorizer.fit_transform(df['Sentence'].values.astype('U')).toarray()



In [72]:
posts

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [73]:
posts.shape

(4930, 5050)

In [74]:
transformed_posts=pd.DataFrame(posts)

In [75]:
df=pd.concat([df,transformed_posts],axis=1)

In [76]:
df.head()

,Sentence,Label,0,1,2,3,4,5,6,7,...,5040,5041,5042,5043,5044,5045,5046,5047,5048,5049
0,a,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,a',1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,a' --,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,a' or 1 = 1; --,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,@,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [77]:
X=df[df.columns[2:]]

In [78]:
X.shape

(4930, 5050)

In [79]:
y=df['Label']

In [80]:
from sklearn.model_selection import train_test_split

In [81]:
# split train test data

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [82]:
X.shape

(4930, 5050)

In [83]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state=0).fit(X_train, y_train)

In [84]:
from sklearn.metrics import accuracy_score

In [85]:
y_pred=clf.predict(X_test)

In [86]:
accuracy_score(y_test, y_pred)

0.9239350912778904

In [ ]:
for i,j in zip(y_test,y_pred):
    print(i==j)

In [88]:
from keras.models import Sequential
from keras import layers
from keras.preprocessing.text import Tokenizer
from keras.wrappers.scikit_learn import KerasClassifier

In [96]:
input_dim = X_train.shape[1]  # Number of features

model = Sequential()

model.add(keras.layers.Dense(1024, input_dim=input_dim, activation='relu'))
model.add(keras.layers.Dense(512, activation='relu'))

model.add(keras.layers.Dense(256, activation='relu'))

model.add(keras.layers.Dense(128, activation='relu'))

# model.add(keras.layers.Dense(20,  activation='relu'))
# model.add(keras.layers.Dense(10,  activation='tanh'))
# # model.add(layers.Flatten())
# model.add(keras.layers.Dense(1024, activation='relu'))

model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(1, activation='sigmoid'))





In [97]:
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 1024)              5172224   
_________________________________________________________________
dense_7 (Dense)              (None, 512)               524800    
_________________________________________________________________
dense_8 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_9 (Dense)              (None, 128)               32896     
_________________________________________________________________
batch_normalization_2 (Batch (None, 128)               512       
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 1)                

In [98]:
import tensorflow as tf

class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('val_accuracy')>0.97):
            print(" \n Reached 97% + validation accuracy")
            self.model.stop_training=True
callbacks = myCallback()

In [99]:
classifier_nn = model.fit(X_train,y_train,
                    epochs=10,
                    verbose=True,
                    validation_data=(X_test, y_test),
                    batch_size=128,
                    callbacks=[callbacks])


Train on 3944 samples, validate on 986 samples
Epoch 1/10
3944/3944 [==============================] - 6s 1ms/step - loss: 0.2330 - accuracy: 0.9067 - val_loss: 0.3472 - val_accuracy: 0.8945
Epoch 2/10
3944/3944 [==============================] - 5s 1ms/step - loss: 0.1141 - accuracy: 0.9566 - val_loss: 0.2736 - val_accuracy: 0.8884
Epoch 3/10
3944/3944 [==============================] - 5s 1ms/step - loss: 0.0783 - accuracy: 0.9673 - val_loss: 0.2406 - val_accuracy: 0.9077
Epoch 4/10
3944/3944 [==============================] - 5s 1ms/step - loss: 0.0740 - accuracy: 0.9713 - val_loss: 0.1913 - val_accuracy: 0.9158
Epoch 5/10
3944/3944 [==============================] - 5s 1ms/step - loss: 0.0768 - accuracy: 0.9691 - val_loss: 0.1457 - val_accuracy: 0.9229
Epoch 6/10
3944/3944 [==============================] - 5s 1ms/step - loss: 0.0685 - accuracy: 0.9698 - val_loss: 0.1270 - val_accuracy: 0.9655
Epoch 7/10
3944/3944 [==============================] - 5s 1ms/step - loss: 0.0674 - accu

In [100]:
pred=model.predict(X_test)

In [101]:
for i in range(len(pred)):
    if pred[i]>0.5:
        pred[i]=1
    elif pred[i]<=0.5:
        pred[i]=0

In [102]:
accuracy_score(y_test,pred)

0.973630831643002

In [ ]:
for i,j in zip(y_test,pred):
    print(i==j)

#### Saving Model

In [114]:
from keras.models import load_model
import pickle

model.save('my_model_additional_sqli_benign_data.h5')
with open('vectorizer_additional_sqli_benign_data', 'wb') as fin:
    pickle.dump(vectorizer, fin)


### Confusion Matrix
#### Assumption:
1. SQLI attack type is positive

In [106]:
def accuracy_function(tp,tn,fp,fn):
    
    accuracy = (tp+tn) / (tp+tn+fp+fn)
    
    return accuracy

In [107]:
def precision_function(tp,fp):
    
    precision = tp / (tp+fp)
    
    return precision

In [108]:
def recall_function(tp,fn):
    
    recall=tp / (tp+fn)
    
    return recall

In [109]:
def confusion_matrix(truth,predicted):
    
    true_positive = 0
    true_negative = 0
    false_positive = 0
    false_negative = 0
    
    for true,pred in zip(truth,predicted):
        
        if true == 1:
            if pred == true:
                true_positive += 1
            elif pred != true:
                false_negative += 1

        elif true == 0:
            if pred == true:
                true_negative += 1
            elif pred != true:
                false_positive += 1
            
    accuracy=accuracy_function(true_positive, true_negative, false_positive, false_negative)
    precision=precision_function(true_positive, false_positive)
    recall=recall_function(true_positive, false_negative)
    
    return (accuracy,
            precision,
           recall)

In [110]:
accuracy,precision,recall=confusion_matrix(y_test,pred)

In [111]:
print(" Accuracy : {0} \n Precision : {1} \n Recall : {2}".format(accuracy, precision, recall))

 Accuracy : 0.973630831643002 
 Precision : 0.9344262295081968 
 Recall : 0.979381443298969


In [112]:

from sklearn.metrics import precision_score
precision_score(y_test, pred)

0.9344262295081968

In [113]:
from sklearn.metrics import recall_score
recall_score(y_test, pred)


0.979381443298969

### Classifying user data

     This cell is also in file user_Data_predict.py No need to run this cell

In [115]:

import keras
from keras.models import load_model
import pickle

mymodel = load_model('my_model_additional_sqli_benign_data.h5')
myvectorizer = pickle.load(open("vectorizer_additional_sqli_benign_data", 'rb'))




def clean_data(input_val):

    input_val=input_val.replace('\n', '')
    input_val=input_val.replace('%20', ' ')
    input_val=input_val.replace('=', ' = ')
    input_val=input_val.replace('((', ' (( ')
    input_val=input_val.replace('))', ' )) ')
    input_val=input_val.replace('(', ' ( ')
    input_val=input_val.replace(')', ' ) ')
    input_val=input_val.replace('1 ', 'numeric')
    input_val=input_val.replace(' 1', 'numeric')
    input_val=input_val.replace("'1 ", "'numeric ")
    input_val=input_val.replace(" 1'", " numeric'")
    input_val=input_val.replace('1,', 'numeric,')
    input_val=input_val.replace(" 2 ", " numeric ")
    input_val=input_val.replace(' 3 ', ' numeric ')
    input_val=input_val.replace(' 3--', ' numeric--')
    input_val=input_val.replace(" 4 ", ' numeric ')
    input_val=input_val.replace(" 5 ", ' numeric ')
    input_val=input_val.replace(' 6 ', ' numeric ')
    input_val=input_val.replace(" 7 ", ' numeric ')
    input_val=input_val.replace(" 8 ", ' numeric ')
    input_val=input_val.replace('1234', ' numeric ')
    input_val=input_val.replace("22", ' numeric ')
    input_val=input_val.replace(" 8 ", ' numeric ')
    input_val=input_val.replace(" 200 ", ' numeric ')
    input_val=input_val.replace("23 ", ' numeric ')
    input_val=input_val.replace('"1', '"numeric')
    input_val=input_val.replace('1"', '"numeric')
    input_val=input_val.replace("7659", 'numeric')
    input_val=input_val.replace(" 37 ", ' numeric ')
    input_val=input_val.replace(" 45 ", ' numeric ')

    return input_val








def predict_sqli_attack():
    
    repeat=True
    
    beautify=''
    for i in range(20):
        beautify+= "="

    print(beautify) 
    input_val=input("Give me some data to work on : ")
    print(beautify)

    
    if input_val== '0':
        repeat=False
    
    

    input_val=clean_data(input_val)
    input_val=[input_val]



    input_val=myvectorizer.transform(input_val).toarray()

    result=mymodel.predict(input_val)


    print(beautify)
    
    
    if repeat == True:
        
        if result>0.5:
            print("ALERT :::: This can be SQL injection")


        elif result<=0.5:
            print("It seems to be safe")
            
        print(beautify)
            
        predict_sqli_attack()
            
    elif repeat == False:
        print( " Good Bye ")

 



In [2]:
# check for sql injection cheat sheet





predict_sqli_attack()

Give me some data to work on : drop sampletable
ALERT :::: This can be SQL injection
Give me some data to work on : select * from members where username ='admin'--' and password='password
ALERT :::: This can be SQL injection
Give me some data to work on : SELECT /*!32302 1/0, */ 1 FROM tablename
ALERT :::: This can be SQL injection
Give me some data to work on : SELECT * FROM members; DROP members--
ALERT :::: This can be SQL injection
Give me some data to work on : IF (1=1) THEN dbms_lock.sleep(3)
ALERT :::: This can be SQL injection
Give me some data to work on : IF (1=1) THEN dbms_lock.sleep(3); ELSE dbms_lock.sleep(0)
ALERT :::: This can be SQL injection
Give me some data to work on : if ((select user) = 'sa' OR (select user) = 'dbo') select 1 else select 1/0
ALERT :::: This can be SQL injection
Give me some data to work on : SELECT login + '-' + password FROM members
ALERT :::: This can be SQL injection
Give me some data to work on : SELECT CONCAT(login, password) FROM members
ALE

ALERT :::: This can be SQL injection
Give me some data to work on : ';BEGIN DECLARE @rt varchar(8000) SET @rd=':' SELECT @rd=@rd+' '+name FROM syscolumns WHERE id =(SELECT id FROM sysobjects WHERE name = 'MEMBERS') AND name>@rd SELECT @rd AS rd into TMP_SYS_TMP end;--
ALERT :::: This can be SQL injection
Give me some data to work on : SELECT table_name FROM information_schema.tables WHERE table_schema = 'databasename'
ALERT :::: This can be SQL injection
Give me some data to work on : SELECT table_name, column_name FROM information_schema.columns WHERE table_name = 'tablename'
ALERT :::: This can be SQL injection
Give me some data to work on : SELECT * FROM all_tables WHERE OWNER = 'DATABASE_NAME'
ALERT :::: This can be SQL injection
Give me some data to work on : SELECT * FROM all_col_comments WHERE TABLE_NAME = 'TABLE'
ALERT :::: This can be SQL injection
Give me some data to work on : SELECT ID, Username, Email FROM [User]WHERE ID = 1 AND ISNULL(ASCII(SUBSTRING((SELECT TOP 1 name FR

It seems to be safe
Give me some data to work on :  UPDATE users SET pass = '1' where user = 't1' OR 1=1--'
ALERT :::: This can be SQL injection
Give me some data to work on : select * from table where id = 1 AND if((ascii(lower(substring((select user()),$i,1))))!=$s,1,benchmark(200000,md5(now())))
ALERT :::: This can be SQL injection
Give me some data to work on : SELECT BENCHMARK(1000000,MD5('A'));
ALERT :::: This can be SQL injection
Give me some data to work on : id=1 OR SLEEP(25)=0 LIMIT 1--
ALERT :::: This can be SQL injection
Give me some data to work on : id=SELECT SLEEP(25)--
ALERT :::: This can be SQL injection
Give me some data to work on :  /?id=1+union+select+1,2,3/*
ALERT :::: This can be SQL injection
Give me some data to work on :  index.php?id=1/*uni X on*/union/*sel X ect*/select+1,2,3/*
ALERT :::: This can be SQL injection
Give me some data to work on :  /?id=1+un/**/ion+sel/**/ect+1,2,3--
ALERT :::: This can be SQL injection
Give me some data to work on :  SELECT * 

ALERT :::: This can be SQL injection
Give me some data to work on : union select 1,2,unhex(hex(Concat(Column_Name,0x3e,Table_schema,0x3e,table_Name))),4,5 /*!from*/information_schema.columns/*!where*/column_name%20/*!like*/char(37,%20112,%2097,%20115,%20115,%2037)
ALERT :::: This can be SQL injection
Give me some data to work on :  http://victim.com/news.php?id=1+un/**/ion+se/**/lect+1,2,3--
It seems to be safe
Give me some data to work on : news.php?id=1+un/**/ion+se/**/lect+1,2,3--
ALERT :::: This can be SQL injection
Give me some data to work on : 
ALERT :::: This can be SQL injection
Give me some data to work on : news.php?id=1+UNunionION+SEselectLECT+1,2,3--
ALERT :::: This can be SQL injection
Give me some data to work on : id=1+uni%0bon+se%0blect+1,2,3--
ALERT :::: This can be SQL injection
Give me some data to work on : id/1%0b | |%0blpad(first_name,7,1)
ALERT :::: This can be SQL injection
Give me some data to work on : ndex.php?page_id=-15+and+(select 1)=(Select 0xAA[..(add a

Give me some data to work on : AND (ascii(substr((SELECT column1 FROM table1 LIMIT 0,1),1,1))) > 95--
ALERT :::: This can be SQL injection
Give me some data to work on : eat healthy AND IF((SELECT ascii(substr(version(),1,1))) > 53,sleep(10),NULL)--
ALERT :::: This can be SQL injection
Give me some data to work on : weather is awesome AND IF((SELECT version()) LIKE "5%",sleep(10),NULL)-- select shoes
ALERT :::: This can be SQL injection
Give me some data to work on : AND IF(((ascii(substr((SELECT column1 FROM table1 LIMIT 0,1),1,1)))) > 95,sleep(10),NULL)--
ALERT :::: This can be SQL injection
Give me some data to work on : These need to be benign data
It seems to be safe
Give me some data to work on : These need to be benign data
It seems to be safe
Give me some data to work on : These need to be benign data
It seems to be safe
Give me some data to work on : 0
 Good Bye 


In [3]:
# check for normal data





predict_sqli_attack()

Give me some data to work on : wait for me
It seems to be safe
Give me some data to work on : a quick brown fox
It seems to be safe
Give me some data to work on : jumps over the alxy dog
It seems to be safe
Give me some data to work on : lazy dog correct
It seems to be safe
Give me some data to work on : And what time will I be I would like an evening flight on the return flight in 4 days
It seems to be safe
Give me some data to work on : that is nonstop the flight duration is 2 hours and 28 minutes and is priced at $337
It seems to be safe
Give me some data to work on : That sounds very good
It seems to be safe
Give me some data to work on : I would like to fly on Delt
It seems to be safe
Give me some data to work on : select from the following menu
It seems to be safe
Give me some data to work on : his menu is to select some food from it
It seems to be safe
Give me some data to work on : Does that complete your order
It seems to be safe
Give me some data to work on : I'd like to sele